## Hypothesis: Flood experience has a positive effect on preparedness

In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [ ]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df = pd.read_csv(path)
df.head()

,id,state,zipcode,geographic_division,census_region,county,experience,supplies,insured,involved,...,sex,education,race,homeownership,income,rentmortgage,rurality,hazard_weight,geometry,zip_count
0,1,Washington,98033,Pacific,West,King,0.0,0,0,1,...,0.0,4,White,1,87500,1250.0,1,0.255097,POLYGON ((-13608075.601186747 6055357.24645823...,1
1,2,Florida,33950,South Atlantic,South,Charlotte,1.0,0,1,1,...,1.0,5,White,1,175000,3000.0,1,2.836145,POLYGON ((-9139287.781401768 3112400.961508024...,1
2,3,New Jersey,7031,Middle Atlantic,Northeast,Bergen,1.0,0,0,1,...,1.0,1,White,0,47500,1250.0,1,1.165971,POLYGON ((-8254065.171859486 4980899.960122439...,1
3,5,Pennsylvania,19148,Middle Atlantic,Northeast,Philadelphia,1.0,0,1,0,...,0.0,4,White,1,125000,1250.0,1,0.624890,POLYGON ((-8368433.592186097 4851218.465580212...,1
4,7,Florida,32976,South Atlantic,South,Brevard,0.0,1,1,0,...,0.0,2,White,1,20000,250.0,1,0.695075,POLYGON ((-8971557.027329512 3234931.888104651...,1


#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- awareness
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication


In [12]:
model = smf.logit('made_safer ~ experience', data=df).fit()
print(model.summary())

print('likelihood of made_safer implemented when there is no flood experience = ', 1 / (1 + np.exp(-model.params['Intercept'])))
print('likelihood of made_safer implemented when there is flood experience = ', 1 / (1 + np.exp(-(model.params['Intercept']+model.params['experience']))))

print('likelihood of one additional  made_safer with one additaion flood experience = ', np.exp(model.params['experience']))

Optimization terminated successfully.
         Current function value: 0.466982
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             made_safer   No. Observations:                  472
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.02281
Time:                        12:17:59   Log-Likelihood:                -220.42
converged:                       True   LL-Null:                       -225.56
Covariance Type:            nonrobust   LLR p-value:                  0.001337
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7483      0.152    -11.523      0.000      -2.046      -1.451
experience     0.8101      0.

<span style= 'color:lightgreen'>

experience has a strong, significant positive impact on made_safer. 

Model fit is very low. 2.3% of variation in the data is captured my the model

Each additional unit of 

In [ ]:
model = smf.logit('alerts ~ experience', data=jacksonville).fit()
print(model.summary())